In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score as ARI

## pip install partition-networkx
## also 'pip install python-louvain' to get 'community' 
import networkx as nx
import partition_networkx
import community


## Simple block model graph

We generate a simple 1000-node graph with 10 communities of size 100 where
* p_in = 0.1, edge probability for pairs of nodes in the same community
* p_out = 0.025, edge probability for pairs of nodes in different communities


In [2]:
# Graph generation with 10 communities of size 100
commSize = 100
numComm = 10
g = nx.generators.planted_partition_graph(l=numComm, k=commSize, p_in=0.1, p_out=0.025)

## store groud truth communities as 'iterables of sets of vertices'
true_comm = [set(list(range(commSize*i, commSize*(i+1)))) for i in range(numComm)]


## Generate partitions with Louvain and ECG

In [3]:
## run Louvain and ECG:
Louvain = community.best_partition(g)
Ecg = community.ecg(g, ens_size=16, resolution=1.0)

In [4]:
## modularity (w.r.t. original weights for ECG)
print('Modularity with Louvain:',community.modularity(Louvain, g))
print('Modularity with ECG:',community.modularity(Ecg.partition, g))

Modularity with Louvain: 0.16761826944320063
Modularity with ECG: 0.2022198878300814


In [5]:
## compute some graph-aware measure given ground truth communities

# for 'gam' partition are either iterables of sets of vertices or 'dict'
print("Adjusted Graph-Aware Rand Index for Louvain:",g.gam(true_comm, Louvain))
print("Adjusted Graph-Aware Rand Index for ECG:",g.gam(true_comm, Ecg.partition))

print("\nJaccard Graph-Aware for Louvain:",g.gam(true_comm, Louvain, method="jaccard",adjusted=False))
print("Jaccard Graph-Aware for ECG:",g.gam(true_comm, Ecg.partition, method="jaccard",adjusted=False))


Adjusted Graph-Aware Rand Index for Louvain: 0.1633348341602583
Adjusted Graph-Aware Rand Index for ECG: 0.7066922428045695

Jaccard Graph-Aware for Louvain: 0.26434583014537105
Jaccard Graph-Aware for ECG: 0.6664522354454808


In [6]:
## compute the adjusted RAND index 
# it requires iterables over the vertices:
tc = {val:idx for idx,part in enumerate(true_comm) for val in part}
# compute ARI
print("Adjusted non-Graph-Aware Rand Index for Louvain:",ARI(list(tc.values()), list(Louvain.values())))
print("Adjusted non-Graph-Aware Rand Index for ecg:",ARI(list(tc.values()), list(Ecg.partition.values())))


Adjusted non-Graph-Aware Rand Index for Louvain: 0.1262778991500082
Adjusted non-Graph-Aware Rand Index for ecg: 0.600695798951731
